In [11]:
import sys
import numpy as np
import subprocess
import time
import os

import getComps
import makeStdCmp
import runAndKillScale
import makeTritonFile
import copyMatAndF33Files
import makeAndRunOrigen
# some notes:
# FMA=fission matrix
#
#
#
##################################################################
########################### USER INPUT ###########################
##################################################################
"""
  Step 0 - ask yourself:
  Is first step length ok in the input file?
  Is the FMA parameter set to yes?
  Are the NPG and other neutron parameters correct?
"""

# fissionable regions - used for origen later
fissionable_mats = [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116]
fissionable_mats_vols = [6.02834870915574000000E+04]*16

# add addnux dict location
addnuxdictbase = 'addnuxDicts/addnux1Dict.dict'

# TODO add volumes, get fission source shape after monte carlo from MC output, use fission source shape to adjust poewrs in each node.

# ORIGEN information
specific_power = [200]
steplength_days = [5]
origen_predictor_divs = 100 # number of time divs for predictor
origen_corrector_divs = 100 # number of time divs for corrector

# file handles
base_triton = 'triton_base.inp'
origen_base = 'baseOrigenFile.inp'

# nprocs
Nprocs = 48


In [12]:

##################################################################
############################ SETUP ###############################
##################################################################

# get temp dir for transport calculations
# tmpdir = sys.argv[2]
tmpdir = 'tmp'

# get machine file
# machinefile = sys.argv[1]
machinefile = 'kasdakljsdasda'

# scale run handles
print("NPROCS", Nprocs)
print("MACHINEFILE",machinefile)
print("BASE_TRITON",base_triton)



NPROCS 48
MACHINEFILE kasdakljsdasda
BASE_TRITON triton_base.inp


In [13]:

# Get materials at time 0 and append
time_lib = getComps.time_dependent_material_lib()
fh = open(base_triton, 'r')
initial_mats = getComps.get_comps(fh)
fh.close()
time_lib.append_lib(initial_mats, time=0.0, step=int(0), PC_flag='C')

# make a base triton input with flags for stdcmp
base_triton_stdcmp = makeTritonFile.makeTritonFile(base_triton, fissionable_mats)

# make a stdcmp file based on initial materials - automatically move stdcmps for triton_stdcmp to tmp dir
for material_index in fissionable_mats:
  thisfilename = 'StdCmpMix'+str(material_index)
  makeStdCmp.makeStdCmpFromMatLib(outputFilename=thisfilename, material_lib=initial_mats, material_index=material_index, tmpdir=tmpdir)

# scale_run_line = ['scalerte', '-N', str(Nprocs), '-M',machinefile, base_triton_stdcmp, '-T', tmpdir, '-m'] # for cluster.
scale_run_line = ['scalerte', base_triton_stdcmp, '-T', tmpdir, '-m'] # for pc


# start iteration
step_num = 0 # temporary thing

### FOR LOOP STARTS HERE

# set origen temp dir


read comp found! Now starting comp directory.
end comp found! Now finalizing comp directory.
read comp found! Now starting comp directory.
end comp found! Now finalizing comp directory.
Now making standard comp file for material 101  | output= StdCmpMix101
Now making standard comp file for material 102  | output= StdCmpMix102
Now making standard comp file for material 103  | output= StdCmpMix103
Now making standard comp file for material 104  | output= StdCmpMix104
Now making standard comp file for material 105  | output= StdCmpMix105
Now making standard comp file for material 106  | output= StdCmpMix106
Now making standard comp file for material 107  | output= StdCmpMix107
Now making standard comp file for material 108  | output= StdCmpMix108
Now making standard comp file for material 109  | output= StdCmpMix109
Now making standard comp file for material 110  | output= StdCmpMix110
Now making standard comp file for material 111  | output= StdCmpMix111
Now making standard comp file for

In [14]:

################################################
################## PREDICTOR ###################
################################################

# STEP 1 - run transport with base file and then copy files from temp dir
runAndKillScale.runAndKillScale(scale_run_line, base_triton_stdcmp)


Started process with PID: 123722
Now sleeping python for 5 seconds
 *******************************************************************************
 *                                                                             *
 *                                 SCALE 6.3.1                                 *
 *                                -------------                                *
 *                                   2023-02                                   *
 *                                                                             *
 *           SCALE:  A Comprehensive Modeling and Simulation Suite             *
 *                   for Nuclear Safety Analysis and Design                    *
 *                                                                             *
 *                      Reactor and Nuclear Systems Division                   *
 *                        Oak Ridge National Laboratory                        *
 *                                        

In [15]:

std_cmp_label = 'PREDICTOR_step'+str(step_num)
f33_label = 'PREDICTOR_step'+str(step_num)
f33_files, std_cmp_files = copyMatAndF33Files.copy_files_from_temp(tmpdir=tmpdir, fissionable_mats=fissionable_mats, std_cmp_label=std_cmp_label, f33_label=f33_label)
# deleteTempDir(tmpdir=tmpdir) # delete temp dir

# Predictor step of origen - first write origen file for this step.
origen_file_list = []
origen_tmpdirs = []
material_lib_at_this_step = getComps.material_lib()
for fiss_mat_id in fissionable_mats:
  # make origen file, append origen file to list of origen files, append Beginning of step mat composition to the material library.
  mat, file_handle, origen_tmpdir = makeAndRunOrigen.makeOrigenFile(origen_base, fiss_mat_id, f33_label, step_num, steplength_days, origen_predictor_divs, specific_power, predictor_corrector_string='PREDICTOR')
  origen_file_list.append(file_handle)

  if step_num == 0:
    # on step 0, we append the input materials to origen - need a baseline nuclide vector to start depletion.
    # for future steps, use predictor corroector to determine what nuclide vector should be
    material_lib_at_this_step.append_mat_to_lib(mat)

  origen_tmpdirs.append(origen_tmpdir)

# # append material lib to time lib once its fully made.
# time_lib.append_lib(material_lib_at_this_step, time=steplength_days[step_num], step=step_num, PC_flag='C') # labelled C since the BOS comp will come from end of corrector step.

# if step num == 0 - we need to add initial nuclides to addnuxdict - addnux dict holds all the addnux nuclides + nuclides defined in the initial material defs
if step_num == 0:
  addnuxdict = getComps.makeNewAddnuxDict(time_lib.mats_by_steps[0].material_dict, tmpdir, addnuxdictbase)



Copied: _triton_mix0109.f33 from temporary directory as F33_109_PREDICTOR_step0.f33
Copied: StdCmpMix00110_00000 from temporary directory as STD_CMP_110_PREDICTOR_step0
Copied: StdCmpMix00112_00000 from temporary directory as STD_CMP_112_PREDICTOR_step0
Copied: StdCmpMix116 from temporary directory as STD_CMP_11_PREDICTOR_step0
Copied: StdCmpMix105 from temporary directory as STD_CMP_10_PREDICTOR_step0
Copied: StdCmpMix00113_00000 from temporary directory as STD_CMP_113_PREDICTOR_step0
Copied: StdCmpMix00105_00000 from temporary directory as STD_CMP_105_PREDICTOR_step0
Copied: _triton_mix0102.f33 from temporary directory as F33_102_PREDICTOR_step0.f33
Copied: StdCmpMix00115_00000 from temporary directory as STD_CMP_115_PREDICTOR_step0
Copied: StdCmpMix108 from temporary directory as STD_CMP_10_PREDICTOR_step0
Copied: StdCmpMix00108_00000 from temporary directory as STD_CMP_108_PREDICTOR_step0
Copied: _triton_mix0103.f33 from temporary directory as F33_103_PREDICTOR_step0.f33
Copied: St

In [ ]:

# run origen now, unpack isotopics, then append isotopics to time_lib
for idx, fiss_mat_id in enumerate(fissionable_mats):
  # get temp directory for this origen directory
  temp = origen_tmpdirs[idx]
  # get origen input
  origen_inp = origen_file_list[idx]
  # run origen
  origen_output_loc = makeAndRunOrigen.runOrigenFile(tmpdir=temp, origen_file=origen_inp, material_id=fiss_mat_id, skipRunning=False)
  origen_f33_loc = temp+'/F33_'+str(fiss_mat_id)+'_'+f33_label+'.f33'
  origen_f71_loc = temp+'/PREDICTOR_EOS_step'+str(step_num)+'_mat'+str(fiss_mat_id)+'.f71'

  # now unpack isotopes from output using output location
  stdCmpFilename = 'StdCmpMix'+str(fiss_mat_id)
  temperature = time_lib.mats_by_steps[0].material_dict[fiss_mat_id].temp # assumes constant temperature

  _ = makeStdCmp.makeStdCmpFromF71(materialNumber=fiss_mat_id, temperature=temperature, filename=origen_f71_loc, dictionaryFilename=addnuxdict, outputFilename=stdCmpFilename, tmpdir=tmpdir, pctag='PREDICTOR')

# now, we have to run eigenvalue calc after depleting, then the predictor step will be completed.

print(temp)
print(origen_output_loc)
print(origen_f33_loc)
print(origen_f71_loc)

 *******************************************************************************
 *                                                                             *
 *                                 SCALE 6.3.1                                 *
 *                                -------------                                *
 *                                   2023-02                                   *
 *                                                                             *
 *           SCALE:  A Comprehensive Modeling and Simulation Suite             *
 *                   for Nuclear Safety Analysis and Design                    *
 *                                                                             *
 *                      Reactor and Nuclear Systems Division                   *
 *                        Oak Ridge National Laboratory                        *
 *                                                                             *
 *                          

rm: cannot remove 'StdCmpMix101PREDICTOR': No such file or directory


np-237 101 0 6.03922704E-34 900 end
pu-238 101 0 2.61518573E-29 900 end
pu-239 101 0 3.45082965E-30 900 end
pu-240 101 0 1.47101082E-27 900 end
pu-241 101 0 1.69243115E-27 900 end
pu-242 101 0 5.22224325E-27 900 end
u-234 101 0 6.34385283E-26 900 end
u-235 101 0 2.03715605E-26 900 end
u-236 101 0 3.96855609E-26 900 end
u-238 101 0 7.89891418E-26 900 end
ENDING LINE
 *******************************************************************************
 *                                                                             *
 *                                 SCALE 6.3.1                                 *
 *                                -------------                                *
 *                                   2023-02                                   *
 *                                                                             *
 *           SCALE:  A Comprehensive Modeling and Simulation Suite             *
 *                   for Nuclear Safety Analysis and Design      

rm: cannot remove 'StdCmpMix102PREDICTOR': No such file or directory


np-237 102 0 2.15632123E-34 900 end
pu-238 102 0 7.15566036E-30 900 end
pu-239 102 0 9.50739803E-31 900 end
pu-240 102 0 1.36913913E-27 900 end
pu-241 102 0 1.52775848E-27 900 end
pu-242 102 0 4.09498510E-27 900 end
u-234 102 0 6.48699481E-26 900 end
u-235 102 0 2.15730539E-26 900 end
u-236 102 0 3.61179931E-26 900 end
u-238 102 0 7.29067456E-26 900 end
ENDING LINE
 *******************************************************************************
 *                                                                             *
 *                                 SCALE 6.3.1                                 *
 *                                -------------                                *
 *                                   2023-02                                   *
 *                                                                             *
 *           SCALE:  A Comprehensive Modeling and Simulation Suite             *
 *                   for Nuclear Safety Analysis and Design      

...

rm: cannot remove 'StdCmpMix114PREDICTOR': No such file or directory


cp $INPDIR/F33_115_PREDICTOR_step0.f33 $TMPDIR/F33_115_PREDICTOR_step0.f33
rm $INPDIR/F33_115_PREDICTOR_step0.f33
rm $INPDIR/STD_CMP_115_PREDICTOR_step0
rm: cannot remove '/home/jfaulkner31/projects/SCALETools/SCALEDepleter/STD_CMP_115_PREDICTOR_step0': No such file or directory
ERROR: System call through shell was not successful (user provided: 'rm $INPDIR/STD_CMP_115_PREDICTOR_step0' translated to 'rm $INPDIR/STD_CMP_115_PREDICTOR_step0')
  Now executing origen                          
***Warning:  The flux over a step has changed by more than 20%.
***Warning:  The flux over a step has changed by more than 20%.
Scale job /home/jfaulkner31/projects/SCALETools/SCALEDepleter/ORIGEN_PREDICTOR_step0_mat115.inp is finished.
Output is stored in /home/jfaulkner31/projects/SCALETools/SCALEDepleter/ORIGEN_PREDICTOR_step0_mat115.out
Now making standard comp file for material 115  | output= StdCmpMix115PREDICTOR
                     NOW RUNNING F71 CONVERSIONS 
 +-------------------------------

rm: cannot remove 'StdCmpMix115PREDICTOR': No such file or directory


u-234 115 0 5.69248361E-26 900 end
u-235 115 0 1.94019992E-26 900 end
u-236 115 0 3.18850476E-26 900 end
u-238 115 0 6.29615680E-26 900 end
ENDING LINE
 *******************************************************************************
 *                                                                             *
 *                                 SCALE 6.3.1                                 *
 *                                -------------                                *
 *                                   2023-02                                   *
 *                                                                             *
 *           SCALE:  A Comprehensive Modeling and Simulation Suite             *
 *                   for Nuclear Safety Analysis and Design                    *
 *                                                                             *
 *                      Reactor and Nuclear Systems Division                   *
 *                        Oak Ridge Na

rm: cannot remove 'StdCmpMix116PREDICTOR': No such file or directory


In [ ]:
print('temp_origen |', temp)
print('output file |'    , origen_output_loc)
print('f33         |'        , origen_f33_loc)
print('f71         |', origen_f71_loc)

In [17]:

# remove some random leftover files from origen:
makeAndRunOrigen.removePattern('F33*.f33')
makeAndRunOrigen.removePattern('ORIGEN_*.f33')
makeAndRunOrigen.removePattern('*.f71')
makeAndRunOrigen.removePattern('ORIGEN_*.msg')


In [18]:
# debugging only -
makeAndRunOrigen.removePattern('ORIGEN_PREDICTOR_step0_mat*.inp')
makeAndRunOrigen.removePattern('STD_CMP_*')
makeAndRunOrigen.removePattern('triton*msg')
makeAndRunOrigen.removePattern('triton*path')
makeAndRunOrigen.removePattern('triton*out')
makeAndRunOrigen.removePattern('tmp*')
makeAndRunOrigen.removePattern('StdCmpMix1*_')